# Mobile App Data Analysis

This notebook aims to analyse a dataset from the Apple iOS app store containing information about roughly 7200 mobile applications. The aim of this project is to determine what kinds of apps attract the most users. It is intended that insights gleamed from this data will inform the company strategy for development of free mobile apps with the aim of improving user in-app ad engagement in order to maximise revenues.

## Data Extraction

The steps in this section are geared towards extracting the data to be analysed into this python notebook. 

In [3]:
from csv import reader
open_apple_file = open('AppleStore.csv', encoding="utf8")
open_android_file = open('googleplaystore.csv', encoding="utf8")

read_apple = reader(open_apple_file)
read_android = reader(open_android_file)


In the above code cell the `open()` funtion is called on both the _Android_ and _Apple iOS_ datasets. The datasets are then extracted via the `reader()` function.

In [4]:
apple_data = list(read_apple)
android_data = list(read_android)


In the above code cell the `list()` function is used to convert both the _Android_ and _Apple iOS_ datasets into list format. 

In [5]:
apple_header = apple_data[0]
android_header = android_data[0]


The above code cell stores the `headers` of each dataset in a variable for easily accessible future reference. Below, the header is removed from the datasets for ease of use in functions.

In [6]:
apple_data = apple_data[1:]
android_data = android_data[1:]


Below a function named `explore_data()` is defined. The purpose of the function is to take in a `dataset` and slice it according to the provided `start` and `end` parameters. It then `prints` a new line for each row or list of information within the dataset. A `rows_and_columns` parameter with its default parameter set to `False` is included which, when set to true, prints the number of rows and columns of the entire dataset. 

In [7]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))


explore_data(apple_data, 0, 1, True)
print("\n")
explore_data(android_data, 0, 1, True)


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
Number of rows: 7197
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
Number of rows: 10841
Number of columns: 13


In [8]:
print(android_header)
print(apple_header)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


## Data Cleaning

The steps in this section are geared towards preparing the data for analysis by sorting relevant data from the non-essential data and fixing any underlying errors present in the lists. 

In [9]:
def check_row_len(dataset):
    faulty_rows = {}
    for row in dataset:
        if len(row) != len(dataset[0]):
            row_index = dataset.index(row)
            faulty_rows[row_index] = (row, len(row))
    return faulty_rows


faulty_android_rows = check_row_len(android_data)
faulty_apple_rows = check_row_len(apple_data)

print(faulty_android_rows)
print(faulty_apple_rows)


{10472: (['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'], 12)}
{}


In the above code cell both app datasets are checked for errors. The _Apple iOS_ dataset has no issues. The _Android_ dataset has one. In the [discussion page](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) of the _Android_ dataset one row is reported to have a missing column. The `check_row_len()` function is used to detect rows that have column errors. The function returns an dictionary named `faulty_rows` with the `row` index value as a key as well as the row itself as a list and its length as a interger in a `tuple`.

Although, the instruction from the guided project outlines on Dataquest was to delete this row from the _Android_ app dataset, in the [discussion page](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015) for this dataset it is mentioned that the missing data falls under the `Category` column. It has also been noted that this app falls under the `Lifestyle` category. I have here opted to add the missing data by creating a new list and appending this information to the relevent index position (2) for the row using the `.append()` method. This corrected row is then used to replace the row with missing data in the original dataset.

In [10]:
corrected_android_10472 = []
corrected_android_10472.append(android_data[10472][0])
corrected_android_10472.append("Lifestyle")
corrected_android_10472 += android_data[10472][1:]

print(corrected_android_10472)

android_data[10472] = corrected_android_10472

print(check_row_len(android_data))


['Life Made WI-Fi Touchscreen Photo Frame', 'Lifestyle', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
{}


The code cell below checks for duplicates within either of the datasets. It stores the names of apps with multiple instances in the `duplicate_apps` list. The function has two parameters. The first parameter,  `dataset`; accepts a dataset in the form of a list of lists as an argument, the other, `names_column`; accepts an integer representing the index value of the _name_ column in the respective dataset's row.  

In [11]:
def check_duplicates(dataset, name_column):

    duplicate_apps = []
    unique_apps = []

    for app in dataset:
        name = app[name_column]

        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)

    return (unique_apps, duplicate_apps)


apple_check = check_duplicates(apple_data, 0)
android_check = check_duplicates(android_data, 0)

apple_dups = apple_check[1]
android_dups = android_check[1]

print("Number of Apple iOS app store dataset duplicate entries:", len(apple_dups))
print("Number of Android app store dataset duplicate entries:", len(android_dups))


Number of Apple iOS app store dataset duplicate entries: 0
Number of Android app store dataset duplicate entries: 1181


As part of cleaning each dataset, duplicates are to be removed. However, entries will not be removed randomly. The entry with the highest tally in the `Reviews` column will be retained as this is the entry that is likey to be the most recent.  

In [12]:
def unique_dict(dataset, name_col, reviews_col, exp):
    reviews_max = {}
    for entry in dataset:
        name = entry[name_col]
        n_reviews = float(entry[reviews_col])
        if name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
        elif name not in reviews_max:
            reviews_max[name] = n_reviews

    if len(reviews_max) == exp:
        return reviews_max
    else:
        return "Dictionary Error: Unexpected Length"


android_exp = len(android_data) - len(android_dups)

android_dict = unique_dict(android_data, 0, 3, android_exp)
print(type(android_dict))


<class 'dict'>


In the above code cell the function `unique_dict()` returns a dictionary of unique app entries (as keys) with their highest rating / review count (as values). It accepts four parameters: the `dataset` as a list of lists, `name_col` and `review_col` as integers, which are the index values for the name and review / ratings columms respectively and `exp` which accepts the expected length of the returned dictionary as an integer. The argument for the `exp` parameter is calculated as the length of the original dataset minus the amount of duplicate entries for that particular dataset. If the dictionary length is not exactly equal to the argumennt for `exp`, `unique_dict()` returns an error message. 

In [13]:
def del_duplicates(dataset, name_col, review_col, u_dict):
    clean_list = []
    dump = []
    for app in dataset:
        name = app[name_col]
        n_reviews = float(app[review_col])
        max_review = u_dict[name]
        if name not in dump and n_reviews == max_review:
            clean_list.append(app)
            dump.append(name)
    return clean_list


android_data = del_duplicates(android_data, 0, 3, android_dict)
print(len(android_data))


9660


The `del_duplicates()` function returns `android_clean`, a list of lists with length equivalent to `exp` meaning the dataset has been successfully cleared of duplicates. 

Both the _Android_ and _Apple i0S_ datasets contain non-English speaker orientated applications. Since the report is being written for a company that is only targeting an English speaking audience, information from applications that target speakers of any language other than english will not be useful in providing insights for this market. The following steps are geared towards removing these apps.   

In [14]:
def check_app_lang(dataset, name_col):
    eng_lang_apps = []
    for app in dataset:
        name = app[name_col]
        char_count = 0

        for char in name:
            if ord(char) > 127:
                char_count += 1

        if char_count < 3:
            eng_lang_apps.append(app)

    return eng_lang_apps


test_dataset = [['Instagram', 0], ['爱奇艺PPS -《欢乐颂2》电视剧热播', 1],
                ['Docs To Go™ Free Office Suite', 2], ['Instachat 😜', 3]]
test_output = check_app_lang(test_dataset, 0)

print(test_output)


[['Instagram', 0], ['Docs To Go™ Free Office Suite', 2], ['Instachat 😜', 3]]


In the above code cell the function `check_app_lang()` attempts to detect potentially non-English speaker orientated apps by scanning the app name to find non-English characters in its name. 

It has two paramaters, one for the `dataset` and the second, `name_col` for the index value associated with the name column of the input dataset. The `check_app_lang()` function iterates over each list and stores the app `name` in a variable. Another for loop uses the `ord()` function to iterate over the `name` string to determine the ASCII code for each charater. 

Each character has a unique ASCII code. The ASCII codes for characters used in the English language fall primarily between 0 and 127. Any outliers are recorded, each outlier per `name` adds one (`+= 1`) to the value of a character count variable (`char_count`) which is initialized with a value of `0`. If the `char_count` value for `name` is less than three (`< 3`), the row of data for that particular app gets added to the `eng_lang_apps` list. 

Once all apps are iterated over, the function returns a list of lists that includes data for all the potentially English speaking audience orientated apps. 

In [15]:
android_data = check_app_lang(android_data, 0)
apple_data = check_app_lang(apple_data, 1)


print(apple_data[:3])
print(len(apple_data))
print("\n")
print(android_data[:3])
print(len(android_data))


[['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']]
6155


[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']]
9598


In [16]:
def isolate_free(dataset, price_col):
    free_apps = []
    for app in dataset:
        price = app[price_col]
        if price == "0":
            free_apps.append(app)
        elif price == "0.0":
            free_apps.append(app)
    return free_apps


print(apple_header)
apple_data = isolate_free(apple_data, 4)
print(apple_data[:3])
print(len(apple_data))
print("\n")
print(android_header)
android_data = isolate_free(android_data, 7)
print(android_data[:3])
print(len(android_data))


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
[['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']]
3203


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

In [17]:
def freq_table(dataset, index):
    table = {}
    for row in dataset:
        key = row[index]
        if key in table:
            table[key] += 1
        elif key not in table:
            table[key] = 1
    return table


The `freq_table` function returns a frequency table based on the inputs of a column as `index` integer and a `dataset` as a list of lists.  

In [18]:
def display_table(dataset, index):
    
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
    
    

The `display_table` function takes in two parameters: `dataset` and `index`, where `dataset` will be a list of lists, and `index` will be an integer. It generates a frequency table using the `freq_table()` function and transforms the the frequency table into a list of tuples and sorts the list in decending order. 

In [19]:
display_table(android_data, 1)


FAMILY : 1676
GAME : 858
TOOLS : 748
BUSINESS : 407
PRODUCTIVITY : 345
LIFESTYLE : 344
FINANCE : 328
MEDICAL : 313
SPORTS : 300
PERSONALIZATION : 294
COMMUNICATION : 286
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 189
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 123
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 71
WEATHER : 70
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 54
BEAUTY : 53
Lifestyle : 1


In [20]:
display_table(android_data, 9)


Tools : 747
Entertainment : 538
Education : 474
Business : 407
Productivity : 345
Lifestyle : 343
Finance : 328
Medical : 313
Sports : 306
Personalization : 294
Communication : 286
Action : 274
Health & Fitness : 273
Photography : 261
News & Magazines : 248
Social : 236
Travel & Local : 206
Shopping : 199
Books & Reference : 189
Simulation : 181
Dating : 165
Arcade : 163
Video Players & Editors : 157
Casual : 156
Maps & Navigation : 123
Food & Drink : 110
Puzzle : 100
Racing : 88
Role Playing : 83
Libraries & Demo : 83
Auto & Vehicles : 82
Strategy : 81
House & Home : 71
Weather : 70
Events : 63
Adventure : 59
Comics : 53
Beauty : 53
Art & Design : 53
Parenting : 44
Card : 40
Trivia : 37
Casino : 37
Educational;Education : 35
Board : 34
Educational : 33
Education;Education : 30
Word : 23
Casual;Pretend Play : 21
Music : 18
Racing;Action & Adventure : 15
Puzzle;Brain Games : 15
Entertainment;Music & Video : 15
Casual;Brain Games : 12
Casual;Action & Adventure : 12
Arcade;Action & Advent

In [21]:
apple_freq_table = freq_table(apple_data, 11)
display_table(apple_data, 11)


Games : 1866
Entertainment : 251
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 83
Utilities : 79
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 50
News : 43
Travel : 40
Finance : 35
Weather : 28
Food & Drink : 26
Reference : 17
Business : 17
Book : 12
Navigation : 6
Medical : 6
Catalogs : 4


In [22]:
def find_popular_apps(dataset, freq_table, genre, ratings_column):

    ratings_table = {}

    for key in freq_table:

        total = 0
        len_genre = 0

        for row in dataset:

            app_genre = row[genre]

            if key == app_genre:
                len_genre += 1
                total += float(row[ratings_column])

            if len_genre > 0:
                avg_ratings = total / len_genre

            ratings_table[key] = avg_ratings

    return ratings_table


The `find_popular_apps` function is designed to take in a frequency table, a `ratings` column (denoting number of ratings per app, not the actual app rating value) and a `genre` colum. The function finds the amount of apps within a particular genre in order to calculate the mean value of ratings per app in the catagory. It outputs a dictionary with the `genre` as key and `mean ratings` as value. 

In [23]:
find_popular_apps(apple_data, apple_freq_table, 11, 5)


{'Social Networking': 71548.34905660378,
 'Photo & Video': 28441.54375,
 'Games': 22886.36709539121,
 'Music': 57326.530303030304,
 'Reference': 79350.4705882353,
 'Health & Fitness': 23298.015384615384,
 'Weather': 52279.892857142855,
 'Utilities': 19156.493670886077,
 'Travel': 28243.8,
 'Shopping': 27230.734939759037,
 'News': 21248.023255813954,
 'Navigation': 86090.33333333333,
 'Lifestyle': 16815.48,
 'Entertainment': 14195.358565737051,
 'Food & Drink': 33333.92307692308,
 'Sports': 23008.898550724636,
 'Book': 46384.916666666664,
 'Finance': 32367.02857142857,
 'Education': 7003.983050847458,
 'Productivity': 21028.410714285714,
 'Business': 7491.117647058823,
 'Catalogs': 4004.0,
 'Medical': 612.0}

In an attempt to analyse the `Android App Store Data` for popular apps, in the code cell below, a frequency table is created that shows the number of apps per category of `amount of installs`. The amounts are binned according to ranges. The range of bins goes from `0` (a bin for apps with no installs) to `1,000,000,00+` (a bin for all the apps with more than a billion installs).  

In [24]:
android_freq_table = freq_table(android_data, 5)
display_table(android_data, 5)


1,000,000+ : 1394
100,000+ : 1021
10,000,000+ : 935
10,000+ : 902
1,000+ : 744
100+ : 613
5,000,000+ : 604
500,000+ : 492
50,000+ : 422
5,000+ : 397
10+ : 313
500+ : 287
50,000,000+ : 202
100,000,000+ : 189
50+ : 170
5+ : 70
1+ : 45
500,000,000+ : 24
1,000,000,000+ : 20
0+ : 4
0 : 1


Below, a `frequency table` for app `catagories` from the `Android Appstore` dataset is created.  

In [25]:
android_cat_ft = freq_table(android_data, 1)


In the cell below the `Android App Store` frequency table for `catagories` is fed into the `popular_app_finder` funcion. It works similarly to `find_popular_apps` with the parameter for `genre` taking in the index information for the `catagories` column and the parameter for ratings taking in information for the `installs` column. The data from the installs needs cleaning as the raw data for number of installs is only catagorical at this stage. In order to find a mean value of installs, the catagorical value will be converted to a numerical `float` value after the removal of special characters and the total number of installs per `catagory` will be calculated based on it. 

In [39]:
def popular_app_finder(dataset, freq_table, category, installs):

    category_table = {}
    
    for key in freq_table:

        total = 0
        len_cat = 0

        for row in dataset:

            app_category = row[category]

            install_num = row[installs]

            install_num = install_num.replace("+", "")
            install_num = install_num.replace(",", "")

            if key == app_category:
                len_cat += 1
                total += float(install_num)

            if len_cat > 0:
                avg_installs = total / len_cat

            category_table[key] = avg_installs

    return category_table


In [40]:
popular_app_finder(android_data, android_cat_ft, 1, 5)

{'ART_AND_DESIGN': 1986335.0877192982,
 'AUTO_AND_VEHICLES': 647317.8170731707,
 'BEAUTY': 513151.88679245283,
 'BOOKS_AND_REFERENCE': 8814199.78835979,
 'BUSINESS': 1712290.1474201474,
 'COMICS': 832613.8888888889,
 'COMMUNICATION': 38590581.08741259,
 'DATING': 854028.8303030303,
 'EDUCATION': 1833495.145631068,
 'ENTERTAINMENT': 11640705.88235294,
 'EVENTS': 253542.22222222222,
 'FINANCE': 1387692.475609756,
 'FOOD_AND_DRINK': 1924897.7363636363,
 'HEALTH_AND_FITNESS': 4188821.9853479853,
 'HOUSE_AND_HOME': 1360598.042253521,
 'LIBRARIES_AND_DEMO': 638503.734939759,
 'LIFESTYLE': 1446158.2238372094,
 'GAME': 15544014.51048951,
 'FAMILY': 3695641.8198090694,
 'MEDICAL': 120550.61980830671,
 'SOCIAL': 23253652.127118643,
 'SHOPPING': 7036877.311557789,
 'PHOTOGRAPHY': 17840110.40229885,
 'SPORTS': 3650602.276666667,
 'TRAVEL_AND_LOCAL': 13984077.710144928,
 'TOOLS': 10830251.970588235,
 'PERSONALIZATION': 5201482.6122448975,
 'PRODUCTIVITY': 16787331.344927534,
 'PARENTING': 542603.62

Notebook still needs analysis for Android App with highest mean installs and Apple App with highest mean ratings as well as recommendations on suggested app ideas for development. Formatting for presentation and redability is also required.